In [ ]:
"""
read in the MVPA data
bring it into the right shape to feed it into the CNN

first do it with one MVPA component
then do it with all MVPA components by concatenating them into a dense layer


"""

In [46]:
import nibabel as nib
import nilearn as nil
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

ImportError: cannot import name 'module_util' from 'tensorflow.python.tools' (unknown location)

In [9]:
### prepare the MVPA data
file_path = "X:/MasterThesis_Reto/Denoised_Data_6mm"
path_content = os.listdir(os.path.join(file_path, "MVPA_data"))

# make two lists with pre (Condition002) and post (Condition003) data of first component
comp1_pre = sorted([x for x in path_content 
                    if "Component001" in x 
                    and "Condition002" in x])
comp1_post = sorted([x for x in path_content 
                    if "Component001" in x 
                    and "Condition003" in x])

print(comp1_pre[:5])

['BETA_Subject001_Condition002_Measure002_Component001.nii', 'BETA_Subject002_Condition002_Measure002_Component001.nii', 'BETA_Subject003_Condition002_Measure002_Component001.nii', 'BETA_Subject004_Condition002_Measure002_Component001.nii', 'BETA_Subject005_Condition002_Measure002_Component001.nii']


In [29]:
# create a dataset with the difference of pre and post data
comp1_diff = []
for pre, post in zip(comp1_pre, comp1_post):
    pre_vol = nib.load(os.path.join(file_path, "MVPA_data", pre))
    post_vol = nib.load(os.path.join(file_path, "MVPA_data", post))
    pre_vol_data = pre_vol.get_fdata()
    post_vol_data = post_vol.get_fdata()
    diff_vol_data = post_vol_data - pre_vol_data
    comp1_diff.append(diff_vol_data)

# check the shape of the data
print(comp1_diff[0].shape)

# check the type of the data
print(type(comp1_diff[0]))

# takes about 3m to run on Dell

(91, 109, 91)
<class 'numpy.ndarray'>


In [30]:
# stack the data to later use it as input for the CNN
# note: the first dimension is the number of samples
inpt_comp1_diff = np.stack(comp1_diff, axis=0)
print(inpt_comp1_diff.shape)

(68, 91, 109, 91)


In [43]:
# read in the excel-file with the labels
label_path = "X:/MasterThesis_Reto"
label_file = "Conn_IDs_Matching.xlsx"
# read excel with only the first three columns
label_df = (pd.read_excel(os.path.join(label_path, label_file),
                            usecols=[0, 1, 2])
            .replace({"Cond": {1: 0}})
            .replace({"Cond": {2: 1}})
            )

label_df[-5:]

,Conn_SubjNr,VPNr,Cond
63,64,80,0
64,65,83,0
65,66,86,1
66,67,87,1
67,68,88,0


In [ ]:
# built a keras CNN model with functional API
inputs = keras.Input(shape=(inpt_comp1_diff.shape[1:]))
x = layers.Conv3D(32, 3, activation="relu")(inputs)
x = layers.MaxPooling3D(3)(x)
x = layers.Conv3D(64, 3, activation="relu")(x)
x = layers.MaxPooling3D(3)(x)
x = layers.Conv3D(128, 3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

# define the model
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

# compile the model
model.compile(optimizer=keras.optimizers.RMSprop(),
                loss=keras.losses.BinaryCrossentropy(),
                metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
# split the data into training and test data
# use 80% of the data for training and 20% for testing
# use cross-validation to get a better estimate of the model performance

x_train, x_test, y_train, y_test = train_test_split(inpt_comp1_diff, label_df["Cond"], test_size=0.2, random_state=42)

cv_scores = cross_val_score(model, inpt_comp1_diff, label_df["Cond"], cv=5)
print(cv_scores)

# fit the model
history = model.fit(inpt_comp1_diff, label_df["Cond"], batch_size=1, epochs=10)

# evaluate the model
model.evaluate(inpt_comp1_diff, label_df["Cond"])

In [ ]:
# plot the training history
plt.plot(history.history["loss"])
plt.plot(history.history["binary_accuracy"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["loss", "accuracy"], loc="upper left")
plt.show()

# save the model
model.save("X:/MasterThesis_Reto/3d_cnn_model")